In [80]:
import pandas as pd
import numpy as np
import os

In [2]:
f = open(os.getcwd() + '/asylum_clean_chicago_eoirstrictconsec3_1.csv')
f.close()

In [77]:
data = pd.read_csv(os.getcwd() + '/asylum_clean_chicago_eoirstrictconsec3_1.csv', low_memory=False)

In [31]:
f2 = open(os.getcwd() + '/column_names.txt', 'r+')

In [4]:
data[["democrat", "republican"]].corr()

,democrat,republican
democrat,1,-1
republican,-1,1


In [50]:
pd.unique(data["DateofAppointment"])

array(['01mar1995', '01dec2010', 'Nov-05', 'Jul-87', nan, '01oct2010',
       'Aug-10', 'Sep-66', '01may2006', '01apr2003', 'Mar-95', 'Aug-89',
       'Apr-97', 'Oct 10', 'Apr-98', '01jul2011', '01oct1984', '01mar1990',
       'Aug-02', '01jun2003', 'Dec 10', 'Sep-95', '01sep1994', 'Apr-06',
       'Mar-93', '01jul1990', 'Nov-09', 'Jun-90', '01apr1997', 'Jan-83',
       'May-06', 'Sep-94', '01feb1976', 'Feb-86', '03jan2003', '01oct1998',
       '01oct1995', 'Mar 12', 'Jan-90', '01may2005', '01feb1994',
       '01jan1991', 'Jun-98', 'Dec-00', '01feb1995', 'Apr-93', '01jun2004',
       '01sep1995', 'May-90', 'Aug-85', '01sep1993', 'Aug-93', 'Mar-08',
       'May-97', 'Oct-94', '01feb2003', '01jun1997', '01sep2006', 'Mar-97',
       '01nov1995', 'May-08', 'Feb-95', 'Jan-87', 'Apr-09', 'Apr-95',
       '01jan1995', '01may1997', '01oct2006', '01jul1995', '01jan2006',
       '01mar1986', 'Jul-95', 'Oct-05', 'Mar-06', '01apr2004', 'Jan-02',
       'Apr-02', 'Jun-04', 'Feb-94', '01feb1982', 'M

In [45]:
datestring = "01nov1995"

In [46]:
import datetime

In [78]:
raw_data = data

In [81]:
# DateofAppointment comes in all different formats so let's transform the dates to a datetime object
raw_data["DateofAppointment_formatted"] = raw_data["DateofAppointment"]
for datestring in pd.unique(raw_data["DateofAppointment"][raw_data["DateofAppointment"].notnull()]):
    try: 
        date = datetime.datetime.strptime(datestring, "%d%b%Y").date()
    except ValueError:
        if datestring[3] == "-":
            date = datetime.datetime.strptime(datestring, "%b-%y").date()
        else:
            date = datetime.datetime.strptime(datestring, "%b %y").date()
    raw_data["DateofAppointment_formatted"][raw_data["DateofAppointment"] == datestring] = date

# comp_date is given in number of days since January 1, 1960, so let's also transform this to a datetime object
dates_available = raw_data["comp_date"][raw_data["comp_date"].notnull()]  
raw_data["comp_date_formatted"] = raw_data["comp_date"]
raw_data["comp_date_formatted"] = [datetime.date(1960, 1, 1) + datetime.timedelta(i) for i in dates_available]

# Now let's create a numeric feature that gives the number of days since the judge's appointment 
# up to the current decision, nan if DateofAppointment is missing for that judge
subset = raw_data[raw_data["comp_date_formatted"].notnull() & raw_data["DateofAppointment_formatted"].notnull()]
timediff = [(a-b).days for (a,b) in zip(subset["comp_date_formatted"], subset["DateofAppointment_formatted"])]
raw_data["TimeSinceAppointment"] = np.nan * np.zeros(len(raw_data))
raw_data["TimeSinceAppointment"][raw_data["comp_date_formatted"].notnull() 
                                 & raw_data["DateofAppointment_formatted"].notnull()] = timediff

In [117]:
data[["Military_Years_SLR"]].describe()

,Military_Years_SLR
count,475549.000000
mean,1.015199
std,4.489096
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,33.000000


In [44]:
sum(data["flag_unknownorderwithinday"])

86844

In [32]:
for col in data.columns:
    f2.write(col +'\n')
f2.close()

In [33]:
data[["courtid", "Court_SLR"]][data["courtid"] == 2]

,courtid,Court_SLR
0,2,SAN DIEGO
1,2,SAN DIEGO
2,2,SAN DIEGO
27379,2,NEW YORK
27380,2,NEW YORK
27381,2,NEW YORK
27382,2,NEW YORK
27383,2,NEW YORK
27384,2,NEW YORK
27385,2,NEW YORK


In [4]:
import sklearn.cross_validation

In [52]:
train, test = sklearn.cross_validation.train_test_split(data)

In [54]:
i = 0
for col in data.columns:
    num_values = len(pd.unique(data[col])) 
    if num_values <=3:
        i += 1
        print ('%s: %s' %(col, pd.unique(data[col])))

lawyer: [1 0]
defensive: [  1.   0.  nan]
written: [0 1]
flag_decisionerror_strdes: [0 1]
flag_decisionerror_idncaseproc: [1 0]
flag_earlystarttime: [0 1]
flag_mismatch_base_city: [0 1]
flag_mismatch_hearing: [0 1]
flag_multiple_proceedings: [1 0]
flag_notfirstproceeding: [1 0]
flag_multiple_proceedings2: [0 1]
grant: [ 1.  0.]
flag_notfirstproceeding2: [1 0]
flag_prevprocgrant: [0 1]
flag_prevprocdeny: [0 1]
flag_unknowntime: [0 1]
flag_unknownorderwithinday: [0]
numfamsperslot: [1]
lastindayD: [0 1]
grantraw: [1 0]
L1grant: [ nan   1.   0.]
L1grant_sameday: [0 1]
L2grant: [ nan   1.   0.]
L2grant_sameday: [0 1]
grant2: [ 1.  0.]
L1grant2: [ nan   1.   0.]
L2grant2: [ nan   1.   0.]
moderategrant3070: [0 1]
moderategrantraw3070: [0 1]
outliermeanyear: [  1.   0.  nan]
outliermeannatyear: [  1.   0.  nan]
outliermeannatdefyear: [ nan   1.   0.]
negoutliermeanyear: [  1.   0.  nan]
negoutliermeannatyear: [  1.   0.  nan]
negoutliermeannatdefyear: [ nan   1.   0.]
moderategrantrawnatdef:

In [9]:
pd.unique(data['defensive'])

array([  1.,   0.,  nan])

In [57]:
data['idncase'].value_counts()

3643221    5
2755615    4
4420422    4
5220275    4
3185520    4
4525664    4
3426943    4
3399811    4
4551906    4
3662894    4
3118828    4
4246477    4
4555255    4
3681525    4
6761801    4
...
2914598    1
3961127    1
3410216    1
5295074    1
4462890    1
2887979    1
6428360    1
4040260    1
3946798    1
3163506    1
6084914    1
2412851    1
5040436    1
5568822    1
3672065    1
Length: 399035, dtype: int64

In [65]:
data[['idncase', 'idnproceeding', 'famcode', 'grant', 'comp_date']][data['idncase'] == 4420422]

,idncase,idnproceeding,famcode,grant,comp_date
279848,4420422,3624810,279847,1,16917
280095,4420422,4401284,280096,0,17870
280177,4420422,5358712,280178,1,18780
280181,4420422,5592037,280182,1,18836


In [3]:
print sum(pd.isnull(data['grant']))
data = data[pd.notnull(data['grant'])]

86844


In [31]:
values = []
for col in data.columns:
    unique_values = len(pd.unique(data[col]))
    values.append((col, unique_values))
col_counts = pd.DataFrame(values, columns = ['column name', 'unique values'])
col_counts.sort(columns='unique values', ascending=False)

,column name,unique values
28,famcode,414209
1,idnproceeding,414209
0,idncase,399035
64,raw_order_court,112329
13,eoirattyid,28057
190,difmeannatyear,16981
191,difmeannatdefyear,16245
193,absdifmeannatyear,15458
194,absdifmeannatdefyear,14738
189,difmeanyear,11977


In [14]:
import numpy as np

In [93]:
data['democrat'].value_counts()

1    242393
0    148549
dtype: int64

In [99]:
sum(data['democrat'].isnull())

23267

In [120]:
np.sqrt(len(data) * .80)

633.12115744144899

In [94]:
data['President_SLR'].value_counts()

Clinton    218070
Reagan      64203
Bush I      41186
Bush II     40045
Obama       12613
Carter       8218
Nixon        3115
Ford         2611
Johnson       881
dtype: int64

In [88]:
1971, 1973, 1974, 1975, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 
1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013

,idncase,idnproceeding,hearing_loc_code,ij_code,comp_date,lawyer,defensive,natid,written,flag_decisionerror_strdes,...,log_military_experience,log_private_experience,log_academic_experience,govD,INSD,militaryD,privateD,academicD,democrat,republican
0,3746462,2113503,AGA,AA,15131,1,1,44,0,0,...,0,2.564949,0,0,0,0,1,0,1,0
1,3635079,1980165,AGA,AA,15131,1,0,44,0,0,...,0,2.564949,0,0,0,0,1,0,1,0
2,3998888,2439804,AGA,AA,15132,1,0,44,0,0,...,0,2.564949,0,0,0,0,1,0,1,0
3,6441123,5102185,FLO,AA,18435,0,1,94,0,0,...,0,2.564949,0,0,0,0,1,0,1,0
4,6443250,5104754,FLO,AA,18436,0,1,54,0,0,...,0,2.564949,0,0,0,0,1,0,1,0
5,6414808,5147864,FLO,AA,18437,0,1,54,0,0,...,0,2.564949,0,0,0,0,1,0,1,0
8,2891398,1198824,LVG,AA,13975,1,0,72,0,0,...,0,2.564949,0,0,0,0,1,0,1,0
9,3569624,1903006,LVG,AA,14010,1,0,72,0,0,...,0,2.564949,0,0,0,0,1,0,1,0
10,3054400,1401304,LVG,AA,14013,1,0,72,0,0,...,0,2.564949,0,0,0,0,1,0,1,0
11,4542988,3141536,PHI,AA,16964,1,1,105,0,0,...,0,2.564949,0,0,0,0,1,0,1,0


In [71]:
data[['ij_code', 'courtid', 'ij_court_code', 'hearing_loc_code', 'year']]

,ij_code,courtid,ij_court_code,hearing_loc_code,year
0,AA,2,1,AGA,2001
1,AA,2,1,AGA,2001
2,AA,2,1,AGA,2001
3,AA,18,2,FLO,2010
4,AA,18,2,FLO,2010
5,AA,18,2,FLO,2010
8,AA,29,6,LVG,1998
9,AA,29,6,LVG,1998
10,AA,29,6,LVG,1998
11,AA,39,7,PHI,2006


In [73]:
data['numfamsperslot'].value_counts()

1     444721
2      42826
3       7788
4       2844
5       1350
6        924
7        301
8        128
9        117
10        30
12        24
dtype: int64

In [75]:
data['numfamsperslot'].value_counts()

1    414209
dtype: int64

In [81]:
data[(data['Male_judge'] == 1) & (data['Gender'] == 'F')]

,idncase,idnproceeding,hearing_loc_code,ij_code,comp_date,lawyer,defensive,natid,written,flag_decisionerror_strdes,...,log_military_experience,log_private_experience,log_academic_experience,govD,INSD,militaryD,privateD,academicD,democrat,republican


In [6]:
len(pd.unique(data['ij_code']))

426

In [5]:
len(pd.unique(data['natid']))

227

In [7]:
len(pd.unique(data['year']))

40

In [4]:
data.shape

(414209, 264)

In [7]:
data.sort(columns='comp_date', ascending=False, inplace=True)

In [11]:
.10*data.shape[0]

41420.9

In [19]:
data.iloc[:7755]['year']

176574    2013
374062    2013
47134     2013
1515      2013
97310     2013
302725    2013
302724    2013
366858    2013
400968    2013
480347    2013
310232    2013
165941    2013
10584     2013
54201     2013
211230    2013
...
60449     2013
168543    2013
389488    2013
127570    2013
388621    2013
455624    2013
47016     2013
419243    2013
278389    2013
105715    2013
135323    2013
189836    2013
190911    2013
64293     2013
92479     2013
Name: year, Length: 7755, dtype: int64

In [17]:
len(data[data['year'] == 2013])

7755